# MediBuddy

Welcome to MediBuddy!
MediBuddy is your Personal Health Companion.
MediBuddy is Capable enough to Identify the Disease you are suffering from just by listing out your Symptoms.
You can also get more information about the disease, its symptoms, precautions, medicautions, diet, workout and symptom-severity-level. 

## Importing Necessary Modules/Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier, ExtraTreesClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

import warnings
warnings.filterwarnings("ignore")

## Loading the Datasets

The folder "data" contains 8 different datasets which are loaded using the Pandas DataFrame.

### Loading the Training Data

In [2]:
df = pd.read_csv('data/Training.csv')

In [3]:
df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [4]:
df.shape

(4920, 133)

### Loading the Symptoms_df

In [5]:
symptom_df = pd.read_csv('data/symtoms_df.csv')

In [6]:
symptom_df.head()

,Unnamed: 0,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4
0,0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches
1,1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN
2,2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN
3,3,Fungal infection,itching,skin_rash,dischromic _patches,NaN
4,4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN


In [7]:
symptom_df.shape

(4920, 6)

In [8]:
symptom_df['Disease'].value_counts()

Disease
Fungal infection                           120
Allergy                                    120
GERD                                       120
Chronic cholestasis                        120
Drug Reaction                              120
Peptic ulcer diseae                        120
AIDS                                       120
Diabetes                                   120
Gastroenteritis                            120
Bronchial Asthma                           120
Hypertension                               120
Migraine                                   120
Cervical spondylosis                       120
Paralysis (brain hemorrhage)               120
Jaundice                                   120
Malaria                                    120
Chicken pox                                120
Dengue                                     120
Typhoid                                    120
hepatitis A                                120
Hepatitis B                                120
Hepat

In [9]:
symptom_df.isnull().sum()

Unnamed: 0      0
Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Symptom_4     348
dtype: int64

In [10]:
symptom_df['Symptom_4'].values[1]

nan

In [11]:
symptom_columns = ['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4']
symptom_long = symptom_df.melt(id_vars='Disease', value_vars=symptom_columns, var_name='Symptom_Level', value_name='Symptom')
symptom_long.dropna(subset=['Symptom'], inplace=True)
symptom_df_new = symptom_long.groupby('Disease')['Symptom'].unique().reset_index()
symptom_df_new.columns = ['Disease', 'Symptoms']
all_unique_symptoms = symptom_long['Symptom'].unique().tolist()

In [12]:
symptom_df_new

,Disease,Symptoms
0,(vertigo) Paroymsal Positional Vertigo,"[ vomiting, headache, nausea, spinning_move..."
1,AIDS,"[ muscle_wasting, patches_in_throat, high_fe..."
2,Acne,"[ skin_rash, pus_filled_pimples, blackheads,..."
3,Alcoholic hepatitis,"[ vomiting, yellowish_skin, abdominal_pain, ..."
4,Allergy,"[ continuous_sneezing, shivering, chills, w..."
5,Arthritis,"[ muscle_weakness, stiff_neck, swelling_join..."
6,Bronchial Asthma,"[ fatigue, cough, high_fever, breathlessnes..."
7,Cervical spondylosis,"[ back_pain, weakness_in_limbs, neck_pain, ..."
8,Chicken pox,"[itching, skin_rash, fatigue, lethargy, hi..."
9,Chronic cholestasis,"[itching, vomiting, yellowish_skin, nausea,..."


In [13]:
symptom_df_new['Symptoms'] = symptom_df_new['Symptoms'].apply(lambda sym_list: [s.replace('_', ' ') for s in sym_list])

In [14]:
symptom_df_new

,Disease,Symptoms
0,(vertigo) Paroymsal Positional Vertigo,"[ vomiting, headache, nausea, spinning move..."
1,AIDS,"[ muscle wasting, patches in throat, high fe..."
2,Acne,"[ skin rash, pus filled pimples, blackheads,..."
3,Alcoholic hepatitis,"[ vomiting, yellowish skin, abdominal pain, ..."
4,Allergy,"[ continuous sneezing, shivering, chills, w..."
5,Arthritis,"[ muscle weakness, stiff neck, swelling join..."
6,Bronchial Asthma,"[ fatigue, cough, high fever, breathlessnes..."
7,Cervical spondylosis,"[ back pain, weakness in limbs, neck pain, ..."
8,Chicken pox,"[itching, skin rash, fatigue, lethargy, hi..."
9,Chronic cholestasis,"[itching, vomiting, yellowish skin, nausea,..."


### Loading the Symptom-severity Data 

In [15]:
symptom_severity = pd.read_csv('data/Symptom-severity.csv')

In [16]:
symptom_severity.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [17]:
symptom_severity.shape

(133, 2)

### Loading the Precautions_df

In [18]:
precautions = pd.read_csv('data/precautions_df.csv')

In [19]:
precautions.head()

,Unnamed: 0,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [20]:
precautions.shape

(41, 6)

### Loading the Workout_df

In [21]:
workout = pd.read_csv('data/workout_df.csv')

In [22]:
workout.head()

,Unnamed: 0.1,Unnamed: 0,disease,workout
0,0,0,Fungal infection,Avoid sugary foods
1,1,1,Fungal infection,Consume probiotics
2,2,2,Fungal infection,Increase intake of garlic
3,3,3,Fungal infection,Include yogurt in diet
4,4,4,Fungal infection,Limit processed foods


In [23]:
workout.shape

(410, 4)

In [24]:
workout['disease'].values[0]

'Fungal infection'

In [25]:
workout_rows = []
for i in range(0, len(workout), 10):
    disease = workout['disease'].iloc[i]
    workout_list = workout['workout'].iloc[i:i+10].tolist()
    workout_rows.append({'disease': disease,'workout': workout_list})
workout_new = pd.DataFrame(workout_rows)

In [26]:
workout_new.head()

,disease,workout
0,Fungal infection,"[Avoid sugary foods, Consume probiotics, Incre..."
1,Allergy,"[Avoid allergenic foods, Consume anti-inflamma..."
2,GERD,"[Consume smaller meals, Avoid trigger foods (s..."
3,Chronic cholestasis,"[Consume a low-fat diet, Eat high-fiber foods,..."
4,Drug Reaction,"[Discontinue offending medication, Stay hydrat..."


### Loading the Description Data

In [27]:
description = pd.read_csv('data/description.csv')

In [28]:
description.head()

,Disease,Description
0,Fungal infection,Fungal infection is a common skin condition ca...
1,Allergy,Allergy is an immune system reaction to a subs...
2,GERD,GERD (Gastroesophageal Reflux Disease) is a di...
3,Chronic cholestasis,Chronic cholestasis is a condition where bile ...
4,Drug Reaction,Drug Reaction occurs when the body reacts adve...


In [29]:
description.shape

(41, 2)

### Loading the Medications Data

In [30]:
medications = pd.read_csv('data/medications.csv')

In [31]:
medications.head()

,Disease,Medication
0,Fungal infection,"['Antifungal Cream', 'Fluconazole', 'Terbinafi..."
1,Allergy,"['Antihistamines', 'Decongestants', 'Epinephri..."
2,GERD,"['Proton Pump Inhibitors (PPIs)', 'H2 Blockers..."
3,Chronic cholestasis,"['Ursodeoxycholic acid', 'Cholestyramine', 'Me..."
4,Drug Reaction,"['Antihistamines', 'Epinephrine', 'Corticoster..."


In [32]:
medications.shape

(41, 2)

### Loading the Diets Data

In [33]:
diets = pd.read_csv('data/diets.csv')

In [34]:
diets.head()

,Disease,Diet
0,Fungal infection,"['Antifungal Diet', 'Probiotics', 'Garlic', 'C..."
1,Allergy,"['Elimination Diet', 'Omega-3-rich foods', 'Vi..."
2,GERD,"['Low-Acid Diet', 'Fiber-rich foods', 'Ginger'..."
3,Chronic cholestasis,"['Low-Fat Diet', 'High-Fiber Diet', 'Lean prot..."
4,Drug Reaction,"['Antihistamine Diet', 'Omega-3-rich foods', '..."


In [35]:
diets.shape

(41, 2)

## Investigating the Training Dataset

In [36]:
df.describe()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
count,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,...,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000
mean,0.137805,0.159756,0.021951,0.045122,0.021951,0.162195,0.139024,0.045122,0.045122,0.021951,...,0.021951,0.021951,0.021951,0.023171,0.023171,0.023171,0.023171,0.023171,0.023171,0.023171
std,0.344730,0.366417,0.146539,0.207593,0.146539,0.368667,0.346007,0.207593,0.207593,0.146539,...,0.146539,0.146539,0.146539,0.150461,0.150461,0.150461,0.150461,0.150461,0.150461,0.150461
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 133 entries, itching to prognosis
dtypes: int64(132), object(1)
memory usage: 5.0+ MB


In [38]:
df.isnull().sum()

itching                 0
skin_rash               0
nodal_skin_eruptions    0
continuous_sneezing     0
shivering               0
                       ..
inflammatory_nails      0
blister                 0
red_sore_around_nose    0
yellow_crust_ooze       0
prognosis               0
Length: 133, dtype: int64

## Splitting of Dataset

Splitting the Training Data into X and y to make it ready for Training the Models.

In [39]:
X = df.drop('prognosis', axis=1)
y = df['prognosis']

In [40]:
X

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4916,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4918,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,1,1,1,0,0,0


In [41]:
y

0                              Fungal infection
1                              Fungal infection
2                              Fungal infection
3                              Fungal infection
4                              Fungal infection
                         ...                   
4915    (vertigo) Paroymsal  Positional Vertigo
4916                                       Acne
4917                    Urinary tract infection
4918                                  Psoriasis
4919                                   Impetigo
Name: prognosis, Length: 4920, dtype: object

## Encoding of Categorical Columns

In [42]:
encoder = LabelEncoder()

In [43]:
temp_df = pd.DataFrame(y)

In [44]:
temp_df.columns = ['prognosis']

In [45]:
temp_df

,prognosis
0,Fungal infection
1,Fungal infection
2,Fungal infection
3,Fungal infection
4,Fungal infection
...,...
4915,(vertigo) Paroymsal Positional Vertigo
4916,Acne
4917,Urinary tract infection
4918,Psoriasis


In [46]:
y = encoder.fit_transform(df['prognosis'])

In [47]:
y

array([15, 15, 15, ..., 38, 35, 27], shape=(4920,))

In [48]:
temp_df['prognosis_encoded'] = pd.DataFrame(y)

In [49]:
temp_df

,prognosis,prognosis_encoded
0,Fungal infection,15
1,Fungal infection,15
2,Fungal infection,15
3,Fungal infection,15
4,Fungal infection,15
...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,0
4916,Acne,2
4917,Urinary tract infection,38
4918,Psoriasis,35


In [50]:
temp_df.drop_duplicates(keep='first', inplace=True)

In [51]:
temp_df

,prognosis,prognosis_encoded
0,Fungal infection,15
10,Allergy,4
20,GERD,16
30,Chronic cholestasis,9
40,Drug Reaction,14
50,Peptic ulcer diseae,33
60,AIDS,1
70,Diabetes,12
80,Gastroenteritis,17
90,Bronchial Asthma,6


In [52]:
a=[]
for i in df.columns:
    if i != 'prognosis':
        a.append(i)

In [53]:
a

['itching',
 'skin_rash',
 'nodal_skin_eruptions',
 'continuous_sneezing',
 'shivering',
 'chills',
 'joint_pain',
 'stomach_pain',
 'acidity',
 'ulcers_on_tongue',
 'muscle_wasting',
 'vomiting',
 'burning_micturition',
 'spotting_ urination',
 'fatigue',
 'weight_gain',
 'anxiety',
 'cold_hands_and_feets',
 'mood_swings',
 'weight_loss',
 'restlessness',
 'lethargy',
 'patches_in_throat',
 'irregular_sugar_level',
 'cough',
 'high_fever',
 'sunken_eyes',
 'breathlessness',
 'sweating',
 'dehydration',
 'indigestion',
 'headache',
 'yellowish_skin',
 'dark_urine',
 'nausea',
 'loss_of_appetite',
 'pain_behind_the_eyes',
 'back_pain',
 'constipation',
 'abdominal_pain',
 'diarrhoea',
 'mild_fever',
 'yellow_urine',
 'yellowing_of_eyes',
 'acute_liver_failure',
 'fluid_overload',
 'swelling_of_stomach',
 'swelled_lymph_nodes',
 'malaise',
 'blurred_and_distorted_vision',
 'phlegm',
 'throat_irritation',
 'redness_of_eyes',
 'sinus_pressure',
 'runny_nose',
 'congestion',
 'chest_pain',


In [54]:
len(a)

132

In [55]:
symptoms_dict = {}
for i in range(0,132):
    symptoms_dict[a[i]] = i

In [56]:
symptoms_dict

{'itching': 0,
 'skin_rash': 1,
 'nodal_skin_eruptions': 2,
 'continuous_sneezing': 3,
 'shivering': 4,
 'chills': 5,
 'joint_pain': 6,
 'stomach_pain': 7,
 'acidity': 8,
 'ulcers_on_tongue': 9,
 'muscle_wasting': 10,
 'vomiting': 11,
 'burning_micturition': 12,
 'spotting_ urination': 13,
 'fatigue': 14,
 'weight_gain': 15,
 'anxiety': 16,
 'cold_hands_and_feets': 17,
 'mood_swings': 18,
 'weight_loss': 19,
 'restlessness': 20,
 'lethargy': 21,
 'patches_in_throat': 22,
 'irregular_sugar_level': 23,
 'cough': 24,
 'high_fever': 25,
 'sunken_eyes': 26,
 'breathlessness': 27,
 'sweating': 28,
 'dehydration': 29,
 'indigestion': 30,
 'headache': 31,
 'yellowish_skin': 32,
 'dark_urine': 33,
 'nausea': 34,
 'loss_of_appetite': 35,
 'pain_behind_the_eyes': 36,
 'back_pain': 37,
 'constipation': 38,
 'abdominal_pain': 39,
 'diarrhoea': 40,
 'mild_fever': 41,
 'yellow_urine': 42,
 'yellowing_of_eyes': 43,
 'acute_liver_failure': 44,
 'fluid_overload': 45,
 'swelling_of_stomach': 46,
 'swelle

In [57]:
diseases_list = temp_df.set_index('prognosis_encoded')['prognosis'].to_dict()

In [58]:
diseases_list

{15: 'Fungal infection',
 4: 'Allergy',
 16: 'GERD',
 9: 'Chronic cholestasis',
 14: 'Drug Reaction',
 33: 'Peptic ulcer diseae',
 1: 'AIDS',
 12: 'Diabetes ',
 17: 'Gastroenteritis',
 6: 'Bronchial Asthma',
 23: 'Hypertension ',
 30: 'Migraine',
 7: 'Cervical spondylosis',
 32: 'Paralysis (brain hemorrhage)',
 28: 'Jaundice',
 29: 'Malaria',
 8: 'Chicken pox',
 11: 'Dengue',
 37: 'Typhoid',
 40: 'hepatitis A',
 19: 'Hepatitis B',
 20: 'Hepatitis C',
 21: 'Hepatitis D',
 22: 'Hepatitis E',
 3: 'Alcoholic hepatitis',
 36: 'Tuberculosis',
 10: 'Common Cold',
 34: 'Pneumonia',
 13: 'Dimorphic hemmorhoids(piles)',
 18: 'Heart attack',
 39: 'Varicose veins',
 26: 'Hypothyroidism',
 24: 'Hyperthyroidism',
 25: 'Hypoglycemia',
 31: 'Osteoarthristis',
 5: 'Arthritis',
 0: '(vertigo) Paroymsal  Positional Vertigo',
 2: 'Acne',
 38: 'Urinary tract infection',
 35: 'Psoriasis',
 27: 'Impetigo'}

## Train - Test Split

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [60]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3936, 132), (984, 132), (3936,), (984,))

## Model Training

### Creating Objects of Algorithm Classes

In [61]:
gnb = GaussianNB()
mnb = MultinomialNB(alpha=1.0)
bnb = BernoulliNB(alpha=1.0)
svc = SVC(kernel='linear', C=1.0, random_state=42)
knc = KNeighborsClassifier(n_neighbors=5, weights='uniform')
dtc = DecisionTreeClassifier(max_depth=1, criterion='entropy')
lrc = LogisticRegression(solver='liblinear',penalty='l2', C=1.0, random_state=42)
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
abc = AdaBoostClassifier(n_estimators=100, random_state=42)
bc = BaggingClassifier(n_estimators=100, random_state=42)
etc = ExtraTreesClassifier(n_estimators=100, max_depth=1, random_state=42)
gbdt = GradientBoostingClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(n_estimators=100, random_state=42)

### Creating a Dictionary to Store the Models

In [62]:
clfs = {
    'SVC' : svc,
    'KN' : knc,
    'NB' : mnb,
    'DT' : dtc,
    'LR' : lrc,
    'RF' : rfc,
    'AdaBoost' : abc,
    'BgC' : bc,
    'ETC' : etc,
    'GBDT' : gbdt,
    'xgb' : xgb 
}

### Function to Predict Accuracy and Precision Scores

In [63]:
def train_classifier(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')

    return accuracy, precision

### Looping through the Dictionary for Model Training

In [64]:
for name, clf in clfs.items():
    current_accuracy, current_precision = train_classifier(clf, X_train, y_train, X_test, y_test)
    cm = confusion_matrix(y_test, clf.predict(X_test))
    print(f"{name} - Accuracy: {current_accuracy}, Precision: {current_precision}")
    print("Confusion Matrix:\n")
    print(np.array2string(cm, separator=', '))
    print("\n" + "="*50 + "\n")

SVC - Accuracy: 1.0, Precision: 1.0
Confusion Matrix:

[[18,  0,  0, ...,  0,  0,  0],
 [ 0, 30,  0, ...,  0,  0,  0],
 [ 0,  0, 24, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 26,  0,  0],
 [ 0,  0,  0, ...,  0, 22,  0],
 [ 0,  0,  0, ...,  0,  0, 34]]


KN - Accuracy: 1.0, Precision: 1.0
Confusion Matrix:

[[18,  0,  0, ...,  0,  0,  0],
 [ 0, 30,  0, ...,  0,  0,  0],
 [ 0,  0, 24, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 26,  0,  0],
 [ 0,  0,  0, ...,  0, 22,  0],
 [ 0,  0,  0, ...,  0,  0, 34]]


NB - Accuracy: 1.0, Precision: 1.0
Confusion Matrix:

[[18,  0,  0, ...,  0,  0,  0],
 [ 0, 30,  0, ...,  0,  0,  0],
 [ 0,  0, 24, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 26,  0,  0],
 [ 0,  0,  0, ...,  0, 22,  0],
 [ 0,  0,  0, ...,  0,  0, 34]]


DT - Accuracy: 0.034552845528455285, Precision: 0.0013309969101405012
Confusion Matrix:

[[0, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0],
 ...,
 [0, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, .

## Functions

This Section contains all the Functions to the Predict the Disease, fetch the Description, Precautions, Medications, Diet, Workout, Symptoms of the Disease and the System-Severity-Level 

### For Predicting the Disease

In [65]:
def fetch_prediction(user_symptoms):
    
    input_data = np.zeros(len(symptoms_dict))

    for symptom in user_symptoms:
        if symptom in symptoms_dict:
            input_data[symptoms_dict[symptom]] = 1
        else:
            print(f"Warning: '{symptom}' not found in symptom dictionary!")

    pred_index = svc.predict([input_data])[0]
    return diseases_list[pred_index]

In [90]:
user_symptoms = input("Enter your symptoms separated by ',': ")
symptoms_list = user_symptoms.split(",")
symptoms_list = [sym.strip() for sym in symptoms_list if sym.strip()]
print(symptoms_list)
fetch_prediction(symptoms_list)

['itching', 'skin_rash', 'nodal_skin_eruptions']


'Fungal infection'

### For Fetching Description 

In [67]:
description

,Disease,Description
0,Fungal infection,Fungal infection is a common skin condition ca...
1,Allergy,Allergy is an immune system reaction to a subs...
2,GERD,GERD (Gastroesophageal Reflux Disease) is a di...
3,Chronic cholestasis,Chronic cholestasis is a condition where bile ...
4,Drug Reaction,Drug Reaction occurs when the body reacts adve...
5,Peptic ulcer disease,Peptic ulcer disease involves sores that devel...
6,AIDS,AIDS (Acquired Immunodeficiency Syndrome) is a...
7,Diabetes,Diabetes is a chronic condition that affects h...
8,Gastroenteritis,Gastroenteritis is an inflammation of the stom...
9,Bronchial Asthma,Bronchial Asthma is a respiratory condition ch...


In [68]:
def fetch_description(disease):
    if disease in description['Disease'].values:
        return description[description['Disease'] == disease]['Description'].values[0]
    else:
        return "Description not found for the given disease."

In [69]:
fetch_description('Paralysis (brain hemorrhage)')

'Paralysis (brain hemorrhage) refers to the loss of muscle function due to bleeding in the brain.'

### For Fetching Precautions

In [70]:
precautions

,Unnamed: 0,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths
5,5,GERD,avoid fatty spicy food,avoid lying down after eating,maintain healthy weight,exercise
6,6,Chronic cholestasis,cold baths,anti itch medicine,consult doctor,eat healthy
7,7,hepatitis A,Consult nearest hospital,wash hands through,avoid fatty spicy food,medication
8,8,Osteoarthristis,acetaminophen,consult nearest hospital,follow up,salt baths
9,9,(vertigo) Paroymsal Positional Vertigo,lie down,avoid sudden change in body,avoid abrupt head movment,relax


In [71]:
def fetch_precautions(disease):
    if disease in precautions['Disease'].values:
        fetched_preacautions = precautions[precautions['Disease'] == disease][['Precaution_1','Precaution_2','Precaution_3','Precaution_4']].values[0]
    else:
        return "Precautions not found for the given disease."

    for i in range(len(fetched_preacautions)):
        print(f"Precaution {i+1}: {fetched_preacautions[i]}")

In [72]:
fetch_precautions('Paralysis (brain hemorrhage)')

Precaution 1: massage
Precaution 2: eat healthy
Precaution 3: exercise
Precaution 4: consult doctor


### For Fetching Medications

In [73]:
medications

,Disease,Medication
0,Fungal infection,"['Antifungal Cream', 'Fluconazole', 'Terbinafi..."
1,Allergy,"['Antihistamines', 'Decongestants', 'Epinephri..."
2,GERD,"['Proton Pump Inhibitors (PPIs)', 'H2 Blockers..."
3,Chronic cholestasis,"['Ursodeoxycholic acid', 'Cholestyramine', 'Me..."
4,Drug Reaction,"['Antihistamines', 'Epinephrine', 'Corticoster..."
5,Peptic ulcer disease,"['Antibiotics', 'Proton Pump Inhibitors (PPIs)..."
6,AIDS,"['Antiretroviral drugs', 'Protease inhibitors'..."
7,Diabetes,"['Insulin', 'Metformin', 'Sulfonylureas', 'DPP..."
8,Gastroenteritis,"['Antibiotics', 'Antiemetic drugs', 'Antidiarr..."
9,Bronchial Asthma,"['Bronchodilators', 'Inhaled corticosteroids',..."


In [74]:
def fetch_medications(disease):
    if disease in medications['Disease'].values:
        fetched_medications = medications[medications['Disease'] == disease]['Medication'].values[0]
        fetched_medications_list = eval(fetched_medications)
        for i, med in enumerate(fetched_medications_list):
            print(f"Medication {i+1}: {med}")
    else:
        return "Medication not found for the given disease."

In [75]:
fetch_medications('Paralysis (brain hemorrhage)')

Medication 1: Blood thinners
Medication 2: Clot-dissolving medications
Medication 3: Anticonvulsants
Medication 4: Physical therapy
Medication 5: Occupational therapy


### For Fetching Diet

In [76]:
diets

,Disease,Diet
0,Fungal infection,"['Antifungal Diet', 'Probiotics', 'Garlic', 'C..."
1,Allergy,"['Elimination Diet', 'Omega-3-rich foods', 'Vi..."
2,GERD,"['Low-Acid Diet', 'Fiber-rich foods', 'Ginger'..."
3,Chronic cholestasis,"['Low-Fat Diet', 'High-Fiber Diet', 'Lean prot..."
4,Drug Reaction,"['Antihistamine Diet', 'Omega-3-rich foods', '..."
5,Peptic ulcer disease,"['Low-Acid Diet', 'Fiber-rich foods', 'Ginger'..."
6,AIDS,"['Balanced Diet', 'Protein-rich foods', 'Fruit..."
7,Diabetes,"['Low-Glycemic Diet', 'Fiber-rich foods', 'Lea..."
8,Gastroenteritis,"['Bland Diet', 'Bananas', 'Rice', 'Applesauce'..."
9,Bronchial Asthma,"['Anti-Inflammatory Diet', 'Omega-3-rich foods..."


In [77]:
def fetch_diet(disease):
    if disease in diets['Disease'].values:
        fetched_diet = diets[diets['Disease'] == disease]['Diet'].values[0]
        fetched_diet_list = eval(fetched_diet)
        for i, diet in enumerate(fetched_diet_list):
            print(f"Diet {i+1}: {diet}")
    else:
        return "Diets not found for the given disease."

In [78]:
fetch_diet('Paralysis (brain hemorrhage)')

Diet 1: Heart-Healthy Diet
Diet 2: Low-sodium foods
Diet 3: Fruits and vegetables
Diet 4: Whole grains
Diet 5: Lean proteins


### For Fetching Workout

In [79]:
workout_new

,disease,workout
0,Fungal infection,"[Avoid sugary foods, Consume probiotics, Incre..."
1,Allergy,"[Avoid allergenic foods, Consume anti-inflamma..."
2,GERD,"[Consume smaller meals, Avoid trigger foods (s..."
3,Chronic cholestasis,"[Consume a low-fat diet, Eat high-fiber foods,..."
4,Drug Reaction,"[Discontinue offending medication, Stay hydrat..."
5,Peptic ulcer disease,"[Consume smaller, more frequent meals, Avoid t..."
6,AIDS,"[Follow a balanced and nutritious diet, Includ..."
7,Diabetes,"[Monitor carbohydrate intake, Eat balanced mea..."
8,Gastroenteritis,"[Stay hydrated, Consume clear fluids, Follow t..."
9,Bronchial Asthma,"[Include anti-inflammatory foods, Consume omeg..."


In [80]:
def fetch_workout(disease):
    if disease in workout_new['disease'].values:
        fetched_workout = workout_new[workout_new['disease'] == disease]['workout'].values[0]
        for i, workout in enumerate(fetched_workout):
            print(f"Workout {i+1}: {workout}")
    else:
        return "Workout not found for the given disease."

In [81]:
fetch_workout('Paralysis (brain hemorrhage)')

Workout 1: Follow a balanced and nutritious diet
Workout 2: Include lean proteins
Workout 3: Consume nutrient-rich foods
Workout 4: Stay hydrated
Workout 5: Include healthy fats
Workout 6: Limit sugary foods and beverages
Workout 7: Include antioxidants
Workout 8: Consume foods rich in vitamin K
Workout 9: Consult a healthcare professional
Workout 10: Manage stress


In [82]:
fetch_workout('Fungal infection')

Workout 1: Avoid sugary foods
Workout 2: Consume probiotics
Workout 3: Increase intake of garlic
Workout 4: Include yogurt in diet
Workout 5: Limit processed foods
Workout 6: Stay hydrated
Workout 7: Consume green tea
Workout 8: Eat foods rich in zinc
Workout 9: Include turmeric in diet
Workout 10: Eat fruits and vegetables


### For Fetching Symptoms

In [83]:
symptom_df_new

,Disease,Symptoms
0,(vertigo) Paroymsal Positional Vertigo,"[ vomiting, headache, nausea, spinning move..."
1,AIDS,"[ muscle wasting, patches in throat, high fe..."
2,Acne,"[ skin rash, pus filled pimples, blackheads,..."
3,Alcoholic hepatitis,"[ vomiting, yellowish skin, abdominal pain, ..."
4,Allergy,"[ continuous sneezing, shivering, chills, w..."
5,Arthritis,"[ muscle weakness, stiff neck, swelling join..."
6,Bronchial Asthma,"[ fatigue, cough, high fever, breathlessnes..."
7,Cervical spondylosis,"[ back pain, weakness in limbs, neck pain, ..."
8,Chicken pox,"[itching, skin rash, fatigue, lethargy, hi..."
9,Chronic cholestasis,"[itching, vomiting, yellowish skin, nausea,..."


In [84]:
def fetch_symptoms(disease):
    if disease in symptom_df_new['Disease'].values:
        return symptom_df_new[symptom_df_new['Disease'] == disease]['Symptoms'].values[0]
    else:
        return "Symptoms not found for the given disease."

In [85]:
fetch_symptoms('Paralysis (brain hemorrhage)')

[' vomiting', ' headache', ' weakness of one body side', ' altered sensorium']

### For Fetching Symptom Severity Level

In [86]:
symptom_severity

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5
...,...,...
128,inflammatory_nails,2
129,blister,4
130,red_sore_around_nose,2
131,yellow_crust_ooze,3


In [87]:
def fetch_symptoms_severity_level(symptom_list):
    symptom_list = symptom_list.split(',')
    print(f"Symptoms provided: {symptom_list}")
    total_severity = 0
    count = 0
    
    for s in symptom_list:
        if s in symptom_severity['Symptom'].values:
            severity = symptom_severity[symptom_severity['Symptom'] == s]['weight'].values[0]
            total_severity += severity
            count += 1
        else:
            print(f"Severity not found for: {s}")
    
    if count == 0:
        return "No valid symptoms found."
    
    rounded_value = round(np.float64(total_severity / count), 2)
    
    if rounded_value < 2.99:
        severity_level = "Severity Level: Mild"
    elif rounded_value < 4.99:
        severity_level = "Severity Level: Moderate"
    else:
        severity_level = "Severity Level: Severe"
    
    return rounded_value, severity_level

In [88]:
fetch_symptoms_severity_level('itching,skin_rash,nodal_skin_eruptions')

Symptoms provided: ['itching', 'skin_rash', 'nodal_skin_eruptions']


(np.float64(2.67), 'Severity Level: Mild')

## Saving the Model

In [89]:
pickle.dump(svc,open('models/svc.pkl', 'wb'))